In [39]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import json

In [99]:
def find_specific_lookup(data, search_image, template_name):
    for entry in data:
        if entry["search_image"] == search_image:
            for template in entry["templates"]:
                if template["template"] == template_name:
                    return template
    return None

In [100]:
from matplotlib import image as mpimg

#get image pairs
base_path = os.path.dirname(os.getcwd())

#label path
lbl_path = os.path.join(base_path, 'Data/labels/train_template_matching.json')

#source and query images
s_img_path = os.path.join(base_path, 'Data/map_train/51.99908_4.373749.png')
q_img_path = os.path.join(base_path, 'Data/train_template_matching')

#for now source path is constant
s_img = cv2.imread(s_img_path)

with open(lbl_path, 'r') as file:
    label = json.load(file)

images = []
templates = []
for file in os.listdir(q_img_path):
    if file.endswith(".jpg") or file.endswith(".png") or file.endswith(".jpeg"):
            q_img = cv2.imread(os.path.join(q_img_path, file))
            images.append([q_img[:, :, :3], s_img[:,:,:3]]) 
            gps = find_specific_lookup(label, '51.99908_4.373749.png', file)
            
            templates.append((q_img[:, :, :3], gps))

In [103]:
def extract_features_for_templates(templates, source_image):
    feature_list = []
    label_list = []
    
    sift = cv2.SIFT_create()
    kp_source, des_source = sift.detectAndCompute(source_image, None)
    
    for template, obj in templates:
        kp_template, des_template = sift.detectAndCompute(template, None)
        gps_coords = obj['gps_coords']
        # Match features
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(des_template, des_source, k=2)
        
        # Lowe's ratio test
        good_matches = []
        for m, n in matches:
            if m.distance < 0.75 * n.distance:
                good_matches.append(m)
        
        # Extract matched keypoints
        src_pts = np.float32([kp_template[m.queryIdx].pt for m in good_matches]).reshape(-1, 2)
        dst_pts = np.float32([kp_source[m.trainIdx].pt for m in good_matches]).reshape(-1, 2)
        
        # Flatten and combine features
        src_flat = src_pts.flatten()
        dst_flat = dst_pts.flatten()
        input_features = np.concatenate([src_flat, dst_flat])
        
        # Append to feature list
        feature_list.append(input_features)
        label_list.append(gps_coords)  # GPS coordinates of this template image
    max_len = max(len(features) for features in feature_list)
    # max_len = 468
    padded_features = [np.pad(features, (0, max_len - len(features))) for features in feature_list]

    return np.array(padded_features), np.array(label_list)

In [104]:
X_train, y_train = extract_features_for_templates(templates, s_img)

In [105]:
from tensorflow.keras import layers, models

# Define the model
def build_model(input_size):
    model = models.Sequential([
        layers.Input(shape=(input_size,)),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(2)  # Predict latitude and longitude
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [106]:
print(X_train.shape[1])

504


In [107]:
# Initialize and train the model
model = build_model(X_train.shape[1])
model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.2)

Epoch 1/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 7515.4663 - mae: 61.4331 - val_loss: 665.6019 - val_mae: 19.4431
Epoch 2/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 641.4940 - mae: 19.6394 - val_loss: 377.1762 - val_mae: 15.6795
Epoch 3/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 266.2264 - mae: 12.8829 - val_loss: 320.7064 - val_mae: 14.2588
Epoch 4/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 189.5544 - mae: 10.9163 - val_loss: 245.6886 - val_mae: 12.5996
Epoch 5/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 163.2838 - mae: 9.9916 - val_loss: 194.1861 - val_mae: 11.0751
Epoch 6/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 130.7165 - mae: 9.0518 - val_loss: 215.4926 - val_mae: 11.9324
Epoch 7/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 119.4356 - mae: 8.5485 - val_loss: 145.6314 - val_mae: 9.7390
Epoch 8/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 97.3232 - mae: 7.7821 - val_loss: 143.8472 - val_mae: 9.7346
Epoch 9/500
25/25 ━━━━

In [118]:
#get image pairs
base_path = os.path.dirname(os.getcwd())

#label path
lbl_path = os.path.join(base_path, 'Data/labels/test_template_matching.json')

#source and query images
s_img_path = os.path.join(base_path, 'Data/map_train/51.99908_4.373749.png')
q_img_path = os.path.join(base_path, 'Data/test_template_matching')

#for now source path is constant
s_img = cv2.imread(s_img_path)

with open(lbl_path, 'r') as file:
    label = json.load(file)

images = []
templates = []
for file in os.listdir(q_img_path):
    if file.endswith(".jpg") or file.endswith(".png") or file.endswith(".jpeg"):
            q_img = cv2.imread(os.path.join(q_img_path, file))
            images.append([q_img[:, :, :3], s_img[:,:,:3]]) 
            gps = find_specific_lookup(label, '51.99908_4.373749.png', file)
            
            templates.append((q_img[:, :, :3], gps))

In [121]:
def extract_features_for_test(templates, source_image):
    feature_list = []
    label_list = []
    
    sift = cv2.SIFT_create()
    kp_source, des_source = sift.detectAndCompute(source_image, None)
    
    for template, obj in templates:
        kp_template, des_template = sift.detectAndCompute(template, None)
        gps_coords = obj['gps_coords']
        # Match features
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(des_template, des_source, k=2)
        
        # Lowe's ratio test
        good_matches = []
        for m, n in matches:
            if m.distance < 0.75 * n.distance:
                good_matches.append(m)
        
        # Extract matched keypoints
        src_pts = np.float32([kp_template[m.queryIdx].pt for m in good_matches]).reshape(-1, 2)
        dst_pts = np.float32([kp_source[m.trainIdx].pt for m in good_matches]).reshape(-1, 2)
        
        # Flatten and combine features
        src_flat = src_pts.flatten()
        dst_flat = dst_pts.flatten()
        input_features = np.concatenate([src_flat, dst_flat])
        
        # Append to feature list
        feature_list.append(input_features)
        label_list.append(gps_coords)  # GPS coordinates of this template image
    # max_len = max(len(features) for features in feature_list)
    max_len = 504
    padded_features = [np.pad(features, (0, max_len - len(features))) for features in feature_list]

    return np.array(padded_features), np.array(label_list)

In [122]:
X_test, y_train = extract_features_for_test(templates, s_img)

In [123]:
print(X_test.shape)

(300, 504)


In [129]:
from haversine import haversine, Unit
distance = 0
for i, test_input in enumerate(X_test):
    print(y_train[i])
    # print(test_input.shape)
    point2 = model.predict(test_input.reshape(1, -1))
    print(point2)
    distance += haversine(y_train[i], point2[0], unit=Unit.METERS)
print(len(X_test))
print(distance/len(X_test))

[51.991689  4.375115]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[49.83817    4.2187595]]
[51.991788  4.362447]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[51.253807   4.4347596]]
[51.99178   4.362269]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[[47.062      4.1319656]]
[51.991911  4.380817]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[[51.38058    4.4926515]]
[51.991973  4.368326]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[[55.03424   4.811132]]
[51.99198   4.362729]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[[48.790485   4.3495355]]
[51.991992  4.375029]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[[50.74972    4.6432676]]
[51.99202   4.388936]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[[48.503513  4.178122]]
[51.99203   4.380806]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
[[54.583443   4.7392645]]
[51.992081  4.363168]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[[51.874847  4.667581]]
[51.992156  4.363652]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[[50.566566  4.417961]]
[51.992161  4.38868 ]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
[[

KeyboardInterrupt: 